In [16]:
from pl_bolts.models.self_supervised import resnets
from pytorch_lightning import seed_everything
from data_loaders.data_module import ChestDataModule
from transforms.finetuning import ChestTrainTransforms, ChestValTransforms

from utils.visualisation import showInRow
from torchvision.models.utils import load_state_dict_from_url
import torch

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from models.baseline import BaseLineClassifier
from models.pretraining.cpcv2 import CPCV2Modified
import pytorch_lightning as pl

In [17]:
seed_everything(1234)

1234

In [21]:
model_kwargs = {
    'pretrained': False,
    'encoder_name': "resnet18",
    'learning_rate': 0.0001,
    'num_classes':2,
    'patch_size':32,
    'patch_overlap':16,
    'num_workers': 2}

model = CPCV2Modified(**model_kwargs)
model.finetune = True

classifier = BaseLineClassifier(model, 
                                num_classes=2, 
                                linear=False,
                                learning_rate=3e-5,
                                b1=0.9,
                                b2=0.999)

wandb_logger = WandbLogger(name='baseline_NL_vinbigdata_full_ssl-weights',project='thesis')
checkpoint_callback = ModelCheckpoint(monitor='val_loss', 
                                      dirpath='logs/baseline/vinbigdata/', 
                                      filename='NL-resnet-full-ssl_weights-{epoch:02d}-{val_loss:.4f}')

trainer = pl.Trainer(gpus=1, deterministic=True,
                     logger=wandb_logger, callbacks=[checkpoint_callback])

if torch.cuda.is_available():
    classifier = classifier.cuda()

GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


"batch_size":    32
"data_dir":      
"encoder":       ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [22]:
data_module = ChestDataModule(ds_list=["vinbigdata"], batch_size=16, num_workers=2, balanced=True, train_fraction=1)
data_module.train_transforms = ChestTrainTransforms(height=256)
data_module.val_transforms = ChestValTransforms(height=256)

Loaded datasets: vinbigdata


In [ ]:
trainer.fit(classifier, data_module)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: wandb version 0.10.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name     | Type          | Params
-------------------------------------------
0 | model    | CPCV2Modified | 14.2 M
1 | accuracy | Accuracy      | 0     

  | Name     | Type          | Params
-------------------------------------------
0 | model    | CPCV2Modified | 14.2 M
1 | accuracy | Accuracy      | 0     


Before sampling length:  3000
After sampling length:  3000


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Before sampling length:  12000
After sampling length:  12000
Creating balanced dataloader


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
